In [1]:
#Installing all required libraries to run the file

!pip install findspark

You should consider upgrading via the 'c:\users\aiswa\appdata\local\programs\python\python37-32\python.exe -m pip install --upgrade pip' command.


Data Preporcessing

Data which is stored in Mongo DB as 4 genres is read into spark dataframes and is combined. Tweets are then cleaned and regex is used to extract show names from tweets. Words starting with hashtags, quotes and capital letters were extracted and they substring operation is carried out. Actual titles from Netflix csv is also read and a string matching is done to get exact show names. Fuzzy string matching is done to those words where the direct matching method was not successful. IMDB ratings of all netflix title were also collected via webscraping and data is saved to a csv file. Final dataframe which will be the input to sentiment analysis is also saved to a csv file.

In [2]:
#Libraries added

import findspark
findspark.init()
#findspark.init('/home/aiswarya_nv4/spark-3.0.1-bin-hadoop2.7')

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd
import functools 
import csv


#working_directory = 'jars/*'



In [3]:
## adding jars for spark connection

conf = SparkConf().set("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.0")


In [4]:
##building a connection with mongodb collection tweet_netflix_comedy using pyspark

spark = SparkSession.builder.appName("myApp") \
        .config("spark.mongodb.input.uri", "mongodb://localhost:27017/Twitter.twitter_comedy_netflix(mongodb://localhost:27017/Twitter.twitter_comedy_netflix)") \
        .config("spark.mongodb.output.uri", "mongodb://localhost:27017/Twitter.twitter_comedy_netflix(mongodb://localhost:27017/Twitter.twitter_comedy_netflix)") \
        .getOrCreate()

In [5]:
##Storing Mongodb data from mongo collection to pyspark dataframes

df_comedy_1 = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri", "mongodb://localhost:27017/Twitter.twitter_comedy_netflix").load()
df_comedy_2 = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri", "mongodb://localhost:27017/Twitter.twitter_comedy_rakhi").load()

df_romance_1 = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri", "mongodb://localhost:27017/Twitter.twitter_romance_netflix").load()
df_romance_2 = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri", "mongodb://localhost:27017/Twitter.twitter_romance_rakhi").load()

df_action_1 = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri", "mongodb://localhost:27017/Twitter.twitter_action_netflix").load()
df_action_2 = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri", "mongodb://localhost:27017/Twitter.twitter_action_rakhi").load()

df_horror_1 = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri", "mongodb://localhost:27017/Twitter.twitter_horror_netflix").load()
df_horror_2 = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri", "mongodb://localhost:27017/Twitter.twitter_horror_rakhi").load()


In [6]:
# Removing object_id column as it is not required for further processing

df_comedy_1 = df_comedy_1.select([c for c in df_comedy_1.columns if c not in {'_id'}])
df_comedy_2 = df_comedy_2.select([c for c in df_comedy_2.columns if c not in {'_id'}])
df_romance_1 = df_romance_1.select([c for c in df_romance_1.columns if c not in {'_id'}])
df_romance_2 = df_romance_2.select([c for c in df_romance_2.columns if c not in {'_id'}])
df_action_1 = df_action_1.select([c for c in df_action_1.columns if c not in {'_id'}])
df_action_2 = df_action_2.select([c for c in df_action_2.columns if c not in {'_id'}])
df_horror_1 = df_horror_1.select([c for c in df_horror_1.columns if c not in {'_id'}])
df_horror_2 = df_horror_2.select([c for c in df_horror_2.columns if c not in {'_id'}])


In [7]:
#Union all genre dataframes into one dataframe

def unionAll(dfs):
    return functools.reduce(lambda df1,df2: df1.union(df2.select(df1.columns)), dfs) 

df_shows = unionAll([df_comedy_1, df_comedy_2, df_romance_1, df_romance_2, df_action_1, df_action_2, df_horror_1, df_horror_2])
df_shows.show()

+-------------------+-------------+--------------------+--------------------+-------------------+---------+--------------------+--------------------+
|            created|retweet_count|              source|                text|            user_id|user_lang|       user_location|           user_name|
+-------------------+-------------+--------------------+--------------------+-------------------+---------+--------------------+--------------------+
|2020-10-13 13:20:48|            0| Twitter for Android|If someone propos...|         1223389308|     null|I'm all over the ...|I Am Incorrigible...|
|2020-10-13 13:07:04|            0|  Twitter for iPhone|I need a comedy o...|          187912008|     null|                    |             titania|
|2020-10-13 13:06:26|            0|  Twitter for iPhone|the unicorn on ne...|1037389925807665153|     null|             Georgia|sad dads music li...|
|2020-10-13 12:59:16|            0|  Twitter for iPhone|@AdamMcKola Watch...|          857145361|   

In [8]:
# Select columns which are required for sring extraction

df_tweets = df_shows.select("user_id", "user_name", "text")
df_tweets.show()


+-------------------+--------------------+--------------------+
|            user_id|           user_name|                text|
+-------------------+--------------------+--------------------+
|         1223389308|I Am Incorrigible...|If someone propos...|
|          187912008|             titania|I need a comedy o...|
|1037389925807665153|sad dads music li...|the unicorn on ne...|
|          857145361|       Andrew Bester|@AdamMcKola Watch...|
|1261248262767427591|      everythinggoes|I need to go back...|
|         2656039566|      Film Companion|#TamilFilms Here'...|
|           46815700|          Sohile Ali|ICYMI - Netflix's...|
| 918876789811830784|       Larry Charles|@MertenWaters In ...|
| 963781111292731393|         Rohitsharma|@yamigautam #Ginn...|
|           18060747|       Derek Kompare|As a champion of ...|
|         1198872132|      Latino Leaders|Mexico’s comedy s...|
|1299512754722582534|ProducersGuidePod...|“The Machine” @be...|
|          247134914|                   

In [9]:
# Extract words in text which starts with '#'
shows_1 = df_tweets.withColumn("text", regexp_extract("text","(?<=^|(?<=[^a-zA-Z0-9-_\.]))#([A-Za-z]+[A-Za-z0-9_]+)",1))
shows_1 = shows_1.dropDuplicates()
shows_1.show(141)

# Extract words in text starting with capital letters
shows_2 = df_tweets.withColumn("text", regexp_extract("text","(?<=^|(?<=[^a-zA-Z0-9-_\.]))[A-Z]([A-Za-z]+[A-Za-z0-9_]+)",0))
shows_2 = shows_2.dropDuplicates()
shows_2.show(794)
# Extract words in text which are enclosed in quotes
shows_3 = df_tweets.withColumn("text", regexp_extract("text","(?<=^|(?<=[^a-zA-Z0-9-_\.]))""([A-Za-z]+[A-Za-z0-9_]+)",0))
shows_3 = shows_3.dropDuplicates()
shows_3.show(942)

# Extract words in text starting with capital or small alphabets
shows_4 = df_tweets.withColumn("text", regexp_extract("text","(?<=^|(?<=[^a-zA-Z0-9-_\.]))[A-z]([A-Za-z]+[A-Za-z0-9_]+)",0))
shows_4 = shows_4.dropDuplicates()
shows_4.show(958)

+-------------------+--------------------+--------------------+
|            user_id|           user_name|                text|
+-------------------+--------------------+--------------------+
|           39534860|                Kyri|                    |
| 906597107930451968|        Mary Poppins|                    |
|1039270619479764992|         EAScheppele|                    |
|           39533743|       She Be Votin!|                    |
|1265179591091654656|          HS infoaid|          Soundtrack|
|1227931725600350211|Zer0 (back to edt...|                    |
|         1051532869|                  🐣|                    |
|           30772321|                hope|                    |
|1294695606762401792|     Szopracz Holmez|                    |
| 968731481727209473|                  AJ|                    |
|           34998272|Que22 (Bruce Wayne )|                    |
|1278768120572686336|                Amir|                    |
|          476467586|   David S. Pimpkins

+--------------------+----------------------+--------------------+
|             user_id|             user_name|                text|
+--------------------+----------------------+--------------------+
|  790834084570357760|  HM NAZMUL ALAM SH...|               Movie|
|  921821296266301441|            Jack Calow|           Hungarian|
|            29171554|    Cátia Isabel Silva|                 All|
|  963564431648018433|  Hey Look I’m Glow...|            Watching|
|           404742060|            Ervien Ace|        Carlstanares|
|            21095241|            BOOdini 🕷|                Adam|
|          4867911856|             Ted Takes|                 The|
|          1051532869|                    🐣|                    |
|          1329565416|                   ∩ʍ∩|              Kdrama|
|  823552739971977216|  ♀️ Sal Robins🟥#I...|             Channel|
|            18904578|  The St. Catharine...|                What|
|          1340096856|  Michele (Rebelwhe...|                Just

+--------------------+----------------------+--------------------+
|             user_id|             user_name|                text|
+--------------------+----------------------+--------------------+
|  820854378324226049|               4RWines|         joshuaflynt|
|  790834084570357760|  HM NAZMUL ALAM SH...|               Movie|
| 1009022347930505216|  'Short & Sweet' F...|                  If|
|          3307366619|Euna (유나) F. Park...|       lukaslenniger|
|            29171554|    Cátia Isabel Silva|                 All|
|  963564431648018433|  Hey Look I’m Glow...|            Watching|
|          1163379666|       dzera dattebayo|                  ve|
|           404742060|            Ervien Ace|        Carlstanares|
|          4867911856|             Ted Takes|                 The|
| 1272788920044945408|   C H A Y O ✨ ᴴᵘˢᵗˡᵉʳ|             suggest|
|  823552739971977216|  ♀️ Sal Robins🟥#I...|             Channel|
|            18904578|  The St. Catharine...|                What

+--------------------+--------------------------------+--------------------+
|             user_id|                       user_name|                text|
+--------------------+--------------------------------+--------------------+
|  820854378324226049|                         4RWines|         joshuaflynt|
|  790834084570357760|            HM NAZMUL ALAM SH...|               Movie|
|          3307366619|          Euna (유나) F. Park...|       lukaslenniger|
|            29171554|              Cátia Isabel Silva|                 All|
|  963564431648018433|            Hey Look I’m Glow...|            Watching|
|           404742060|                      Ervien Ace|        Carlstanares|
|          4867911856|                       Ted Takes|                 The|
| 1272788920044945408|             C H A Y O ✨ ᴴᵘˢᵗˡᵉʳ|             suggest|
|  823552739971977216|            ♀️ Sal Robins🟥#I...|             Channel|
|            18904578|            The St. Catharine...|                What|
| 

In [10]:
# Union of all extracted words
unioned_df = unionAll([shows_1, shows_2, shows_3, shows_4])

unioned_df = unioned_df.filter("text != ''")

unioned_df.show()

+-------------------+--------------------+--------------------+
|            user_id|           user_name|                text|
+-------------------+--------------------+--------------------+
|1265179591091654656|          HS infoaid|          Soundtrack|
|           72627052|           TrivWorks|                 SNL|
| 730922550302281729|        🇵🇷 👁 🇵🇷|            VPDebate|
|           14970333|Monsters and Critics|             Netflix|
|          113123861|                Niru|      BlackMoneyLove|
|1266394863463010305|        Sarfi🧚🏻‍♀️|              Terter|
|         2242846435|        Phytoversity|              SOTWPF|
| 713396467898183681|     TV Cancel Beast|       ElfOnTheShelf|
|           72631996| IRS #COVIDreliefIRS|                 IRS|
|          367589500|         Shah Shahid|AvatarTheLastAirb...|
|1048176160809598976|                  Me|              horror|
|         2230707726|        Chris Savage|              horror|
| 960769246799302656|Film Companion South|     

In [11]:
# Please update the path for the Netflix_updated csv file before running this cell

#Read actual Netflix show names from Netflix_updated csv to a pyspark dtataframe

df_netflix = spark.read.csv('Netflix_updated.csv', header=True)

df_netflix_show = df_netflix.select("title", "type", "listed_in", "Genre")
df_netflix_csv = df_netflix.select("title", "type", "listed_in", "Genre")

#df_netflix_show.show()

# Clean title column
def clean_text(c):
    c = lower(c)
    c = regexp_replace(c, " ", "")
    c = regexp_replace(c, ":", "")
    c = regexp_replace(c, "#", "")
    c = regexp_replace(c, ",", "")
    c = regexp_replace(c, "!", "")
    c = regexp_replace(c, "'", "")
    c = regexp_replace(c, "\\s+", "")

  #c = split(c, "\\s+") tokenization...
    return c

#Clean and Drop any duplicate rows for the actual titles
df_netflix_show = df_netflix_show.withColumn("clean_title", clean_text(col("title"))).dropDuplicates()
df_netflix_show.show(100)

#Clean and Drop any duplicate rows for extracted words from text
unioned_df = unioned_df.withColumn("clean_tweet", clean_text(col("text"))).dropDuplicates()
unioned_df.show(100)

+--------------------+-------+--------------------+-------+--------------------+
|               title|   type|           listed_in|  Genre|         clean_title|
+--------------------+-------+--------------------+-------+--------------------+
|Gad Elmaleh: Amer...|  Movie|     Stand-Up Comedy| Comedy|gadelmalehamerica...|
|     Suite Française|  Movie|Dramas, Internati...|Romance|      suitefrançaise|
|          Lion Pride|TV Show|International TV ...|Romance|           lionpride|
|  The Great Magician|  Movie|Action & Adventur...| Action|    thegreatmagician|
|            Outlawed|  Movie|  Action & Adventure| Action|            outlawed|
|              Helios|  Movie|Action & Adventur...| Action|              helios|
|                Hero|  Movie|Action & Adventur...| Action|                hero|
|           Laddaland|  Movie|Horror Movies, In...| Horror|           laddaland|
|        Ragini MMS 2|  Movie|Horror Movies, In...| Horror|          raginimms2|
|        Soul to Keep|  Movi

+-------------------+--------------------+----------------+----------------+
|            user_id|           user_name|            text|     clean_tweet|
+-------------------+--------------------+----------------+----------------+
| 993718700774637568|Butterfly 🦋 🏴󠁧...|  NetflixBayYanl|  netflixbayyanl|
|          108455155|          WLOY Radio|      nowplaying|      nowplaying|
|1287784840469917710|        Harry Hudson|          comedy|          comedy|
|         3199539929|            #ENDSARS|     SARSMUSTEND|     sarsmustend|
|1177250242850254848|Darkest Hour Apparel|    HorrorMovies|    horrormovies|
|1281315480737906694|grayfogboundstrai...|          horror|          horror|
| 988035033058463745|Harold Cutting | ...|ClimateActionNow|climateactionnow|
|1040923886756478977|    Pete The Pumpkin|  HubieHalloween|  hubiehalloween|
|1157115217551097857|  AlaskanHeartHome🐬|      AndrewYang|      andrewyang|
|           17862527| We Are Iowa Local 5|             The|             the|
|  

In [12]:
# Substring the first 8 characters of extracted words from tweets

df_substr = unioned_df.withColumn('substr_tweet', unioned_df.clean_tweet.substr(1, 8))
df_substr.show()   

# Substring the first 8 characters of actual show titles
df_substr_net = (df_netflix_show.withColumn('substr_title', df_netflix_show.clean_title.substr(1, 8)))
df_substr_net.show() 



+-------------------+--------------------+----------------+----------------+------------+
|            user_id|           user_name|            text|     clean_tweet|substr_tweet|
+-------------------+--------------------+----------------+----------------+------------+
| 993718700774637568|Butterfly 🦋 🏴󠁧...|  NetflixBayYanl|  netflixbayyanl|    netflixb|
|          108455155|          WLOY Radio|      nowplaying|      nowplaying|    nowplayi|
|1287784840469917710|        Harry Hudson|          comedy|          comedy|      comedy|
|         3199539929|            #ENDSARS|     SARSMUSTEND|     sarsmustend|    sarsmust|
|1177250242850254848|Darkest Hour Apparel|    HorrorMovies|    horrormovies|    horrormo|
|1281315480737906694|grayfogboundstrai...|          horror|          horror|      horror|
| 988035033058463745|Harold Cutting | ...|ClimateActionNow|climateactionnow|    climatea|
|1040923886756478977|    Pete The Pumpkin|  HubieHalloween|  hubiehalloween|    hubiehal|
|115711521755

In [14]:
#String match the substring columns of the extracted words and actual show titles to find an exact match
import pyspark.sql.functions as f

df_match_str = df_substr.alias("d1").join(df_substr_net.alias("d2"), f.col("d1.substr_tweet") == f.col("d2.substr_title"), "left")


In [14]:
# Drop NA rows after matching 
df_match_str = df_match_str.dropna()
df_match_str.count()

1115

In [15]:
df_match_str.show()

+-------------------+--------------------+--------------------+--------------------+------------+--------------------+-------+--------------------+-------+--------------------+------------+
|            user_id|           user_name|                text|         clean_tweet|substr_tweet|               title|   type|           listed_in|  Genre|         clean_title|substr_title|
+-------------------+--------------------+--------------------+--------------------+------------+--------------------+-------+--------------------+-------+--------------------+------------+
|1151699417407184897|          Rin Rin ³⁹|            Tomorrow|            tomorrow|    tomorrow| Tomorrow Never Dies|  Movie|  Action & Adventure| Action|   tomorrowneverdies|    tomorrow|
|1151699417407184897|          Rin Rin ³⁹|            Tomorrow|            tomorrow|    tomorrow|   Tomorrow with You|TV Show|International TV ...|Romance|     tomorrowwithyou|    tomorrow|
|           24644560|             Richard|        

In [16]:
# Selecting rows from matched dataframe which are exactly matching i.e extracted words == actual title
df_equal = df_match_str.filter(f.col("clean_tweet") == f.col("clean_title"))
df_equal.show()

+-------------------+--------------------+-----------+-----------+------------+-----------+-------+--------------------+-------+-----------+------------+
|            user_id|           user_name|       text|clean_tweet|substr_tweet|      title|   type|           listed_in|  Genre|clean_title|substr_title|
+-------------------+--------------------+-----------+-----------+------------+-----------+-------+--------------------+-------+-----------+------------+
|1305241152182861824|                J ♥️|        Eli|        eli|         eli|        Eli|  Movie|       Horror Movies| Horror|        eli|         eli|
| 894832907851837444|      Dead Celluloid|Rattlesnake|rattlesnake|    rattlesn|Rattlesnake|  Movie|Horror Movies, In...| Horror|rattlesnake|    rattlesn|
|          212220703|    The Illusive Man|      Still|      still|       still|      Still|  Movie|Horror Movies, In...| Horror|      still|       still|
|         2675216486|       Daniel Cannon| TheWitcher| thewitcher|    thewit

In [17]:
# Selecting rows which are not an exact match i.e extract word != actual title
df_notequal = df_match_str.filter(f.col("clean_tweet") != f.col("clean_title"))
df_notequal.show()


+-------------------+--------------------+--------------------+--------------------+------------+--------------------+-------+--------------------+-------+--------------------+------------+
|            user_id|           user_name|                text|         clean_tweet|substr_tweet|               title|   type|           listed_in|  Genre|         clean_title|substr_title|
+-------------------+--------------------+--------------------+--------------------+------------+--------------------+-------+--------------------+-------+--------------------+------------+
|1151699417407184897|          Rin Rin ³⁹|            Tomorrow|            tomorrow|    tomorrow| Tomorrow Never Dies|  Movie|  Action & Adventure| Action|   tomorrowneverdies|    tomorrow|
|1151699417407184897|          Rin Rin ³⁹|            Tomorrow|            tomorrow|    tomorrow|   Tomorrow with You|TV Show|International TV ...|Romance|     tomorrowwithyou|    tomorrow|
|           24644560|             Richard|        

In [18]:
# Count of not equal rows
df_notequal.count()

670

In [19]:
#Fuzzy String Matching : matches the extracted words and actual titles and gives a score based on similarity of strings.
# Lower the score, the string are more similar

df_lev = df_notequal.withColumn("levenshtein", f.levenshtein(f.col("clean_tweet"), f.col("clean_title")))
df_lev.show()

+-------------------+--------------------+--------------------+--------------------+------------+--------------------+-------+--------------------+-------+--------------------+------------+-----------+
|            user_id|           user_name|                text|         clean_tweet|substr_tweet|               title|   type|           listed_in|  Genre|         clean_title|substr_title|levenshtein|
+-------------------+--------------------+--------------------+--------------------+------------+--------------------+-------+--------------------+-------+--------------------+------------+-----------+
|1151699417407184897|          Rin Rin ³⁹|            Tomorrow|            tomorrow|    tomorrow| Tomorrow Never Dies|  Movie|  Action & Adventure| Action|   tomorrowneverdies|    tomorrow|          9|
|1151699417407184897|          Rin Rin ³⁹|            Tomorrow|            tomorrow|    tomorrow|   Tomorrow with You|TV Show|International TV ...|Romance|     tomorrowwithyou|    tomorrow|   

In [20]:
# Selecting only rows which have a match score less than 10

df_lev10 = df_lev.filter("levenshtein < 10")
df_lev10.show()

+-------------------+--------------------+--------------------+--------------------+------------+--------------------+-------+--------------------+-------+--------------------+------------+-----------+
|            user_id|           user_name|                text|         clean_tweet|substr_tweet|               title|   type|           listed_in|  Genre|         clean_title|substr_title|levenshtein|
+-------------------+--------------------+--------------------+--------------------+------------+--------------------+-------+--------------------+-------+--------------------+------------+-----------+
|1151699417407184897|          Rin Rin ³⁹|            Tomorrow|            tomorrow|    tomorrow| Tomorrow Never Dies|  Movie|  Action & Adventure| Action|   tomorrowneverdies|    tomorrow|          9|
|1151699417407184897|          Rin Rin ³⁹|            Tomorrow|            tomorrow|    tomorrow|   Tomorrow with You|TV Show|International TV ...|Romance|     tomorrowwithyou|    tomorrow|   

In [21]:
# Union of fully matched and fuzzy matched dataframes to create a final dataframe

df_final_tweet = unionAll([df_equal, df_lev10])
df_final_tweet.show()

+-------------------+--------------------+-----------+-----------+------------+-----------+-------+--------------------+-------+-----------+------------+
|            user_id|           user_name|       text|clean_tweet|substr_tweet|      title|   type|           listed_in|  Genre|clean_title|substr_title|
+-------------------+--------------------+-----------+-----------+------------+-----------+-------+--------------------+-------+-----------+------------+
|1305241152182861824|                J ♥️|        Eli|        eli|         eli|        Eli|  Movie|       Horror Movies| Horror|        eli|         eli|
| 894832907851837444|      Dead Celluloid|Rattlesnake|rattlesnake|    rattlesn|Rattlesnake|  Movie|Horror Movies, In...| Horror|rattlesnake|    rattlesn|
|          212220703|    The Illusive Man|      Still|      still|       still|      Still|  Movie|Horror Movies, In...| Horror|      still|       still|
|         2675216486|       Daniel Cannon| TheWitcher| thewitcher|    thewit

In [23]:

df_netflix_show.show()

+--------------------+-------+--------------------+-------+--------------------+
|               title|   type|           listed_in|  Genre|         clean_title|
+--------------------+-------+--------------------+-------+--------------------+
|Gad Elmaleh: Amer...|  Movie|     Stand-Up Comedy| Comedy|gadelmalehamerica...|
|     Suite Française|  Movie|Dramas, Internati...|Romance|      suitefrançaise|
|          Lion Pride|TV Show|International TV ...|Romance|           lionpride|
|  The Great Magician|  Movie|Action & Adventur...| Action|    thegreatmagician|
|            Outlawed|  Movie|  Action & Adventure| Action|            outlawed|
|              Helios|  Movie|Action & Adventur...| Action|              helios|
|                Hero|  Movie|Action & Adventur...| Action|                hero|
|           Laddaland|  Movie|Horror Movies, In...| Horror|           laddaland|
|        Ragini MMS 2|  Movie|Horror Movies, In...| Horror|          raginimms2|
|        Soul to Keep|  Movi

In [25]:
# Final considated data from mongodb for all genres
df_shows = df_shows.dropDuplicates()

In [26]:
# Consolidating final dataframe for sentiment analysis with prooper show names and genre assigned to each user.

df_match_tweets = df_shows.alias("d1").join(df_final_tweet.alias("d2"), f.col("d1.user_id") == f.col("d2.user_id"), "inner")
df_match_tweets = df_match_tweets.select("d1.user_id","d2.user_id", "d2.title", "d1.text", "d2.Genre")
df_match_tweets.show()

+-------------------+-------------------+--------------------+--------------------+-------+
|            user_id|            user_id|               title|                text|  Genre|
+-------------------+-------------------+--------------------+--------------------+-------+
|1121629253135556609|1121629253135556609|               Still|🤔 I still count ...| Horror|
|1289101613123039233|1289101613123039233|                 Don|Don’t ever tell m...| Action|
|           30398906|           30398906|            Thriller|iLOVE THE QUALITY...| Horror|
|         3378324215|         3378324215|            Remember|I remember the go...| Action|
|           39584754|           39584754|                 Don|Don’t respond to ...| Action|
|          450893684|          450893684|               Three|Three favourite h...| Action|
|1165779847882452992|1165779847882452992| Tomorrow Never Dies|no it’s tomorrow ...| Action|
|1165779847882452992|1165779847882452992|   Tomorrow with You|no it’s tomorrow ..

In [ ]:
# Read the IMDB rating file and perform substring operation to get only the numerical part of rating.

df_netflix_ratings = spark.read.csv('IMDB_full_rating_.csv', header=True)
df_netflix_ratings.show()

df_netflix_ratings = df_netflix_ratings.withColumn("rating", df_netflix_ratings.IMDB_rating.substr(1,3))
df_netflix_ratings.show()

netflix_ratings = df_netflix_ratings.select("IMDB_title", "rating")

In [ ]:
# Write the transformed IMDB ratings part into a new csv file
netflix_ratings.toPandas().to_csv('mynetflixcsv.csv',index=False) 

In [30]:
#Export final dataframe to csv file for sentiment analysis

df_match_tweets.toPandas().to_csv('group_tweets1.csv',index=False) 